In [1]:
import os, sys
sys.path.insert(1, os.path.join(os.getcwd()  , '../..'))

In [2]:
# imports
import random
import multiprocessing as mp
from collections import defaultdict
from core import * 
import plotly.graph_objects as go

In [3]:
possible_items = [
    TwoDimensionalItem(300, 400),
    TwoDimensionalItem(600, 300),
    TwoDimensionalItem(600, 400),
    TwoDimensionalItem(300, 200),
]

vehicle = Vehicle(
    compartments=[
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 600),
        TwoDimensionalCompartment(800, 600),
    ]
)

In [4]:
# Define some problems

# Problem 1
nb_items_1 = 15
V_1 = list(range(0, nb_items_1 * 2 + 1)) 
random.seed(0)
items_1 = random.choices(possible_items, k=nb_items_1)
random.seed(0)
distance_matrix_1 = [
    [random.randint(1, 100) if i != 0 and j!=0 and i != j else 0 for i in V_1]
    for j in V_1
]
problem_1 = TwoDimensionalProblem(items_1, vehicle, distance_matrix_1)

# Problem 2
nb_items_2 = 15
V_2 = list(range(0, nb_items_2 * 2 + 1)) 
random.seed(100)
items_2 = random.choices(possible_items, k=nb_items_2)
random.seed(100)
distance_matrix_2 = [
    [random.randint(1, 100) if i != 0 and j!=0 and i != j else 0 for i in V_2]
    for j in V_2
]
problem_2 = TwoDimensionalProblem(items_2, vehicle, distance_matrix_2)

In [5]:
# Initial solutions for the problem by running the gurobi solver
initial_solution_time_limit = 1 # 1 second

# Problem 1                 
modelled_problem_1 = ModelledTwoDimensionalProblem(items_1, vehicle)
modelled_problem_1.create_model()
modelled_problem_1.apply_constraints()
modelled_problem_1.set_model_objective()
modelled_problem_1.solve(time_limit=initial_solution_time_limit)
initial_solution_1 = modelled_problem_1.extract_solution()

# Problem 2                 
modelled_problem_2 = ModelledTwoDimensionalProblem(items_2, vehicle)
modelled_problem_2.create_model()
modelled_problem_2.apply_constraints()
modelled_problem_2.set_model_objective()
modelled_problem_2.solve(time_limit=initial_solution_time_limit)
initial_solution_2 = modelled_problem_2.extract_solution()

initial_solutions_dict = {
    problem_1: initial_solution_1,
    problem_2: initial_solution_2,
}

Set parameter Username
Academic license - for non-commercial use only - expires 2022-05-14
Set parameter TimeLimit to value 1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 12819 rows, 1469 columns and 61479 nonzeros
Model fingerprint: 0xf772a26f
Variable types: 0 continuous, 1469 integer (1252 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 8e+02]
  RHS range        [1e+00, 8e+02]
Presolve removed 1255 rows and 298 columns
Presolve time: 0.19s
Presolved: 11564 rows, 1171 columns, 43492 nonzeros
Variable types: 0 continuous, 1171 integer (961 binary)
Found heuristic solution: objective 1159.0000000

Root relaxation: objective 1.397000e+02, 522 iterations, 0.06 seconds (0.05 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumb

In [6]:
# Analyze how parallelism affects PLNS by using different numbers of processes
# across runs and degrees of destructions. For each run, fix the degree of 
# destruction (to fix expected computation per search iteration).
nb_cpus = mp.cpu_count()
nb_processes = list(range(1, nb_cpus)) # max nb_cpus - 1
time_limit = 20
destruction_degree_limits = [0.15, 0.2, 0.3] # to get 2, 3, 4 destruction degrees


In [7]:
from typing import Any, List, Dict

def get_solver_stats_per_process(
    problem: TwoDimensionalProblem, 
    initial_solution: Solution,
    nb_processes_list: List[int], 
    time_limit: float,
    min_degree_percent: float,
    max_degree_percent: float, 
) -> Dict[int, List[Dict[str, Any]]]:
    problem_results = dict()
    
    for nb_proc in nb_processes_list:
        # initialize a solver with the time limit and the number of process
        # for problem 1 with the initial solution of the gurobi solver
        solver = PLNS(
            problem, 
            initial_solution, 
            time_limit=time_limit, 
            nb_processes=nb_proc,
            min_destruction_degree=min_degree_percent,
            max_destruction_degree=max_degree_percent
        )
        solver.set_destruction_degree_criterion(DestructionDegreeCriterion.CONSTANT)
        solver.search()
        problem_results[nb_proc] = [solver.stats]
    
    return problem_results

In [8]:
# Problem 1 & 2 with destruction degree 2 (0.15 % for 15 items)

nb_processes_to_solver_stats_dict_2 = defaultdict(list)

for problem, initial_solution in initial_solutions_dict.items():
    problem_results = get_solver_stats_per_process(
        problem, 
        initial_solution,
        nb_processes,
        time_limit,
        min_degree_percent=0.15,
        max_degree_percent=0.15,    
    )
    for nb_proc in nb_processes:
        nb_processes_to_solver_stats_dict_2[nb_proc].extend(problem_results[nb_proc])


Parallel LNS initialized with 1 processes
Terminating LNS in 131 iterations and obj 288
Parallel LNS terminating w/ objective 288 in 131 iterations in 20.052916765213013 secs
Parallel LNS initialized with 2 processes
Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 110 iterations and obj 260
Parallel LNS terminating w/ objective 260 in 110 iterations in 20.102165937423706 secs
Parallel LNS initialized with 3 processes
Terminating LNS in 111 iterations and obj 234
Parallel LNS terminating w/ objective 234 in 111 iterations in 20.136448860168457 secs
Parallel LNS initialized with 4 processes
Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 109 iterations and obj 285
Parallel LNS terminating w/ objective 285 in 109 iterations in 20.12850570678711 secs
Parallel LNS initialized with 5 processes
Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 114 iterations and obj 277
Parallel LNS termi

In [9]:
# Get iterations time for every cached solution in solver stats
# for every solver stats (problem_1 and problem_2), for every process nb (1..11)

nb_processes_to_iteration_times_2 = {
    nb_proc: [
        sol.iteration_time
        for stats in solver_stats_list for sol in stats["cached_solutions"]
        if sol.iteration_time > 1e-5
    ] 
    for nb_proc, solver_stats_list in nb_processes_to_solver_stats_dict_2.items()
} 

# Box plot for iteration times per number of process for degree of destruction = 2
fig = go.Figure()
for nb_proc, times in nb_processes_to_iteration_times_2.items():
    fig.add_trace(go.Box(y=times, name=nb_proc))

fig.show()


In [10]:
# Repeat for destruction degree 3 (0.2 %)
# Problem 1 & 2 with destruction degree 3 (0.2 % for 15 items)

# Solve
nb_processes_to_solver_stats_dict_3 = defaultdict(list)
for problem, initial_solution in initial_solutions_dict.items():
    problem_results = get_solver_stats_per_process(
        problem, 
        initial_solution,
        nb_processes,
        time_limit,
        min_degree_percent=0.2,
        max_degree_percent=0.2,    
    )
    for nb_proc in nb_processes:
        nb_processes_to_solver_stats_dict_3[nb_proc].extend(problem_results[nb_proc])

Parallel LNS initialized with 1 processes
Terminating LNS in 88 iterations and obj 284
Parallel LNS terminating w/ objective 284 in 88 iterations in 20.148770809173584 secs
Parallel LNS initialized with 2 processes
Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 80 iterations and obj 286
Parallel LNS terminating w/ objective 286 in 80 iterations in 20.038261651992798 secs
Parallel LNS initialized with 3 processes
Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 76 iterations and obj 256
Parallel LNS terminating w/ objective 256 in 76 iterations in 20.168838500976562 secs
Parallel LNS initialized with 4 processes
Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 69 iterations and obj 220
Parallel LNS terminating w/ objective 220 in 69 iterations in 20.193018674850464 secs
Parallel LNS initialized with 5 processes
Terminating LNS in 53 iterations and obj 289
Parallel LNS terminating w

In [11]:
# Get stats and plot
# 
# Get iterations time for every cached solution in solver stats
# for every solver stats (problem_1 and problem_2), for every process nb (1..11)
nb_processes_to_iteration_times_3 = {
    nb_proc: [
        sol.iteration_time
        for stats in solver_stats_list for sol in stats["cached_solutions"]
        if sol.iteration_time > 1e-5
    ] 
    for nb_proc, solver_stats_list in nb_processes_to_solver_stats_dict_3.items()
} 

# Plot
fig = go.Figure()
for nb_proc, times in nb_processes_to_iteration_times_3.items():
    fig.add_trace(go.Box(y=times, name=nb_proc))

fig.show()

In [14]:
# Repeat for destruction degree 4 (0.3 %)
# Problem 1 & 2 with destruction degree 4 (0.3 % for 15 items)

# Solve
nb_processes_to_solver_stats_dict_4 = defaultdict(list)
for problem, initial_solution in initial_solutions_dict.items():
    problem_results = get_solver_stats_per_process(
        problem, 
        initial_solution,
        nb_processes,
        time_limit,
        min_degree_percent=0.3,
        max_degree_percent=0.3,    
    )
    for nb_proc in nb_processes:
        nb_processes_to_solver_stats_dict_4[nb_proc].extend(problem_results[nb_proc])

Parallel LNS initialized with 1 processes
Terminating LNS in 68 iterations and obj 269
Parallel LNS terminating w/ objective 269 in 68 iterations in 20.217440605163574 secs
Parallel LNS initialized with 2 processes
Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 78 iterations and obj 266
Parallel LNS terminating w/ objective 266 in 78 iterations in 20.158053398132324 secs
Parallel LNS initialized with 3 processes
Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 58 iterations and obj 293
Parallel LNS terminating w/ objective 293 in 58 iterations in 20.14935803413391 secs
Parallel LNS initialized with 4 processes
Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 49 iterations and obj 272
Parallel LNS terminating w/ objective 272 in 49 iterations in 20.229610443115234 secs
Parallel LNS initialized with 5 processes
Terminating LNS in 44 iterations and obj 276
Parallel LNS terminating w/

In [16]:
# Get stats and plot
# 
# Get iterations time for every cached solution in solver stats
# for every solver stats (problem_1 and problem_2), for every process nb (1..11)
nb_processes_to_iteration_times_4 = {
    nb_proc: [
        sol.iteration_time
        for stats in solver_stats_list for sol in stats["cached_solutions"]
        if sol.iteration_time > 1e-5
    ] 
    for nb_proc, solver_stats_list in nb_processes_to_solver_stats_dict_4.items()
} 

# Plot
fig = go.Figure()
for nb_proc, times in nb_processes_to_iteration_times_4.items():
    fig.add_trace(go.Box(y=times, name=nb_proc))

fig.show()